# Перенос файла с диска

In [1]:
!cp "/content/drive/MyDrive/GlebHac/hackathon_part_1.mp4" ./

In [2]:
!cp "/content/drive/MyDrive/GlebHac/vox1_meta.csv" ./

In [3]:
!mkdir splited_sound

In [4]:
!pip install pydub

In [5]:
!pip install SpeechRecognition

     |████████████████████████████████| 32.8 MB 38 kB/s 


In [6]:
!pip install Levenshtein

     |████████████████████████████████| 110 kB 4.2 MB/s 
     |████████████████████████████████| 1.6 MB 29.2 MB/s 


In [7]:
!pip install transliterate

     |████████████████████████████████| 45 kB 2.5 MB/s 


# Получение аудио из видео

In [8]:
from moviepy.editor import * #подключаем пакет moviepy

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2326528/45929032 bytes (5.1%)5259264/45929032 bytes (11.5%)8634368/45929032 bytes (18.8%)12107776/45929032 bytes (26.4%)15433728/45929032 bytes (33.6%)18653184/45929032 bytes (40.6%)22052864/45929032 bytes (48.0%)25174016/45929032 bytes (54.8%)28688384/45929032 bytes (62.5%)32251904/45929032 bytes (70.2%)35692544/45929032 bytes (77.7%)39182336/45929032 bytes (85.3%)

In [9]:
audioclip = AudioFileClip("hackathon_part_1.mp4") #видеофайл 1.mp4
audioclip.write_audiofile("out_audio.mp3") #извлеченная аудиодорожка в файл out_audio.mp3

[MoviePy] Writing audio in out_audio.mp3


100%|██████████| 4387/4387 [00:05<00:00, 770.27it/s]

[MoviePy] Done.


# Разделение аудиофайла на кусочки по n сек

In [10]:
AUDIO_FILE = "out_audio.mp3" # имя исходного файла
SEGMENT_LEN = 10 * 1000 # длинна одного отрезка 

In [11]:
from pydub import AudioSegment

In [12]:
sound = AudioSegment.from_file(AUDIO_FILE) # подгружаем файл

In [13]:
for milsec in range(0,len(sound),SEGMENT_LEN):
  segment=sound[milsec:milsec+SEGMENT_LEN] # отделяем сегмент
  segment.export("./splited_sound/s_"+str(milsec)+".wav", format="wav") # закидываем в файл

# Распознование речи


In [14]:
import speech_recognition as sr 
import IPython
from os import listdir

In [15]:
lines=[]

In [16]:
r = sr.Recognizer()

In [17]:
FILE_NAMES=sorted(os.listdir("./splited_sound"))

In [18]:
for name in FILE_NAMES:
  with sr.AudioFile("./splited_sound/"+name) as source:
    audio = r.record(source)
  lines += [r.recognize_google(audio,language="ru")]

# Поиск контрольных слов

In [19]:
import pandas as pd
control_names = [name.split("_")[-1].lower() for name in list(pd.read_csv("vox1_meta.csv",sep="\t")["VGGFace1 ID"])]+["timberlake"]

In [20]:
import Levenshtein
# Функция, вычисляющая схожесть строк по Левенштейну
def text_sim(s1, s2) -> float:
    ratio = Levenshtein.distance(s1, s2)
    return 1.0 - ratio / max(len(s1), len(s2))


# Функция проверяющая, что слово схоже в заданной степени с хотя бы одним из слов
def is_search_word(word, true_words, min_sim=0.6) -> bool:
    word = word.lower()
    is_true_word = False
    for example_word in true_words:
        is_true_word = is_true_word or (text_sim(word, example_word) > min_sim)
    return is_true_word

In [21]:
translit_lines = [0]*len(lines)

In [22]:
from transliterate import translit
for i,line in enumerate(lines):
  translit_lines[i] =  translit(line,  language_code='ru',reversed=True).lower().split()

In [23]:
text_sim("timberland","timberlake")

0.8

In [ ]:
translit_lines

In [25]:
files_dict = dict(zip(FILE_NAMES,translit_lines))

In [26]:
lines

['Всем привет Меня зовут Серёжа Я работаю звуки на позиции Bacardi Да она собака идёт 12 человек и как и любой буканьеры когда работаю Я люблю слушать музыку',
 'чаще всего это Dubstep или Future Garage люблю слушать Боря привет Я Дима я техлит в продуктовой команде которая занимается волнами в сверхзвуке',
 'классно замечательная и контейнер там вообще отлично себя проявил Но больше всего мне нравится камера Александра пистолетова я',
 'на мероприятии которое представляет себя проигрывание в симфоническом оркестре во всяких этих саундтреков из фильмов и',
 'Вот как раз у триммера скучала композиции которая прикольно очень слушать в исполнении большого оркестра там с органом',
 'живым звуком это очень круто звучит вообще уехал как-то в маршрутке слышал Этот трек сам не слушаю',
 'музыку этого жанра нужно есть качественное делаю там mastering вот это всё вполне на уровне Поэтому вот блин Первый раз слышу',
 'очень понравился добавился в коллекцию Сбер звуки в детстве короче слышал её по

In [27]:
finded_names = set()
true_segment_names=set() 

In [28]:
for segment in files_dict.keys():
  for word in files_dict[segment]:
    if is_search_word(word,control_names,0.85):
      finded_names.add(word)
      true_segment_names.add(segment)

In [29]:
true_segment_names

{'s_180000.wav', 's_190000.wav'}

In [30]:
finded_names

{'furtado', 'timberlake'}

In [31]:
from os import mkdir,chdir

In [40]:
lines=[]

In [41]:
for segment_name in true_segment_names:
  segment_name_clear=segment_name.split(".")[0]
  try:
    mkdir(segment_name_clear)
  except: pass
  sound = AudioSegment.from_file("./splited_sound/s_180000.wav") # подгружаем файл
  for milsec in range(0,len(sound),2000):
    segment=sound[milsec:milsec+SEGMENT_LEN] # отделяем сегмент
    segment.export("./"+segment_name_clear+"/"+segment_name_clear+"_"+str(milsec)+".wav", format="wav") # закидываем в файл
  FILE_NAMES=sorted(os.listdir("./"+segment_name_clear))
  for name in FILE_NAMES:
    with sr.AudioFile("./"+segment_name_clear+"/"+segment_name_clear+"_"+str(milsec)+".wav") as source:
      audio = r.record(source)
    lines += [r.recognize_google(audio,language="ru")]

In [42]:
lines

['тоже молчи',
 'тоже молчи',
 'тоже молчи',
 'тоже молчи',
 'тоже молчи',
 'тоже молчи',
 'тоже молчи',
 'тоже молчи',
 'тоже молчи',
 'тоже молчи',
 'тоже молчи',
 'тоже молчи',
 'тоже молчи',
 'тоже молчи',
 'тоже молчи',
 'тоже молчи',
 'тоже молчи',
 'тоже молчи',
 'тоже молчи',
 'тоже молчи']